In [1]:
!pip install -q h5py pyyaml 

In [2]:
from __future__ import absolute_import, division, print_function

import os

import tensorflow as tf
from tensorflow import keras

tf.__version__

'1.10.0'

In [3]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

In [4]:
# Returns a short sequential model
def create_model():
  model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation=tf.nn.relu, input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10, activation=tf.nn.softmax)
  ])
  
  model.compile(optimizer=tf.keras.optimizers.Adam(), 
                loss=tf.keras.losses.sparse_categorical_crossentropy,
                metrics=['accuracy'])
  
  return model


# Create a basic model instance
model = create_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [5]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create checkpoint callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, 
                                                 save_weights_only=True,
                                                 verbose=1)

model = create_model()

model.fit(train_images, train_labels,  epochs = 10, 
          validation_data = (test_images,test_labels),
          callbacks = [cp_callback])  # pass callback to training

Train on 1000 samples, validate on 1000 samples
Epoch 1/10
1000/1000 [==============================] - 11s 11ms/step - loss: 1.2203 - acc: 0.6540 - val_loss: 0.7348 - val_acc: 0.7790

Epoch 00001: saving model to training_1/cp.ckpt
Epoch 2/10
1000/1000 [==============================] - 0s 216us/step - loss: 0.4377 - acc: 0.8830 - val_loss: 0.5258 - val_acc: 0.8350

Epoch 00002: saving model to training_1/cp.ckpt
Epoch 3/10
1000/1000 [==============================] - 0s 198us/step - loss: 0.3010 - acc: 0.9210 - val_loss: 0.4720 - val_acc: 0.8440

Epoch 00003: saving model to training_1/cp.ckpt
Epoch 4/10
1000/1000 [==============================] - 0s 196us/step - loss: 0.2179 - acc: 0.9480 - val_loss: 0.4444 - val_acc: 0.8590

Epoch 00004: saving model to training_1/cp.ckpt
Epoch 5/10
1000/1000 [==============================] - 0s 229us/step - loss: 0.1555 - acc: 0.9700 - val_loss: 0.4026 - val_acc: 0.8720

Epoch 00005: saving model to training_1/cp.ckpt
Epoch 6/10
1000/1000 [=====

In [9]:
#!ls {checkpoint_dir}
!dir {checkpoint_dir}

 Volume in drive D is LENOVO
 Volume Serial Number is 06CB-B159

 Directory of D:\Work Space\Python\training_1

09/04/2018  05:13 PM    <DIR>          .
09/04/2018  05:13 PM    <DIR>          ..
09/04/2018  05:13 PM                71 checkpoint
09/04/2018  05:13 PM         1,631,720 cp.ckpt.data-00000-of-00001
09/04/2018  05:13 PM               647 cp.ckpt.index
               3 File(s)      1,632,438 bytes
               2 Dir(s)  976,584,245,248 bytes free


In [10]:
model = create_model()

loss, acc = model.evaluate(test_images, test_labels)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 494us/step
Untrained model, accuracy:  6.50%


In [11]:
model.load_weights(checkpoint_path)
loss,acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 63us/step
Restored model, accuracy: 87.10%


In [12]:
# include the epoch in the file name. (uses `str.format`)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path, verbose=1, save_weights_only=True,
    # Save weights, every 5-epochs.
    period=5)

model = create_model()
model.fit(train_images, train_labels,
          epochs = 50, callbacks = [cp_callback],
          validation_data = (test_images,test_labels),
          verbose=0)


Epoch 00005: saving model to training_2/cp-0005.ckpt

Epoch 00010: saving model to training_2/cp-0010.ckpt

Epoch 00015: saving model to training_2/cp-0015.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00025: saving model to training_2/cp-0025.ckpt

Epoch 00030: saving model to training_2/cp-0030.ckpt

Epoch 00035: saving model to training_2/cp-0035.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00045: saving model to training_2/cp-0045.ckpt

Epoch 00050: saving model to training_2/cp-0050.ckpt


In [13]:
import pathlib

# Sort the checkpoints by modification time.
checkpoints = pathlib.Path(checkpoint_dir).glob("*.index")
checkpoints = sorted(checkpoints, key=lambda cp:cp.stat().st_mtime)
checkpoints = [cp.with_suffix('') for cp in checkpoints]
latest = str(checkpoints[-1])
checkpoints

[WindowsPath('training_2/cp-0030.ckpt'),
 WindowsPath('training_2/cp-0035.ckpt'),
 WindowsPath('training_2/cp-0040.ckpt'),
 WindowsPath('training_2/cp-0045.ckpt'),
 WindowsPath('training_2/cp-0050.ckpt')]

In [14]:
model = create_model()
model.load_weights(latest)
loss, acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 109us/step
Restored model, accuracy: 87.40%


In [15]:
# Save the weights
model.save_weights('./checkpoints/my_checkpoint')

# Restore the weights
model = create_model()
model.load_weights('./checkpoints/my_checkpoint')

loss,acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 120us/step
Restored model, accuracy: 87.40%


In [16]:
model = create_model()

model.fit(train_images, train_labels, epochs=5)

# Save entire model to a HDF5 file
model.save('my_model.h5')

Epoch 1/5
1000/1000 [==============================] - 0s 449us/step - loss: 1.1581 - acc: 0.6760
Epoch 2/5
1000/1000 [==============================] - 0s 125us/step - loss: 0.4206 - acc: 0.8800
Epoch 3/5
1000/1000 [==============================] - 0s 125us/step - loss: 0.2902 - acc: 0.9260
Epoch 4/5
1000/1000 [==============================] - 0s 129us/step - loss: 0.2086 - acc: 0.9470
Epoch 5/5
1000/1000 [==============================] - 0s 118us/step - loss: 0.1687 - acc: 0.9590


In [17]:
# Recreate the exact same model, including weights and optimizer.
new_model = keras.models.load_model('my_model.h5')
new_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_6 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [18]:
loss, acc = new_model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 160us/step
Restored model, accuracy: 85.50%
